![Memories Teach](https://lh3.google.com/u/2/d/11o4JyBYhHcmz-LSt63GpOs4lfW9j39T_=w1912-h954-iv1)


`Basic to Advance in Google colab's for image processing, pattern recognition and computer vision`

[Phonepaserth SISAYKEO]

Reference: visioncolab

# Model Selector

These examples show the functions necessary to implement model selection strategies, i.e. strategies to choose the best feature selector and the best classifier. [(See more in Spanish)](https://github.com/domingomery/patrones/blob/master/clases/Cap05_Evaluacion/presentations/PAT05_ModelSelector.pdf)

## Setup

In [ ]:
import numpy as np
from   sklearn.svm import SVC
from   sklearn.model_selection import train_test_split
from   sklearn.metrics import confusion_matrix, accuracy_score
from   sklearn.model_selection import cross_val_score
from   sklearn.model_selection import StratifiedKFold,KFold,LeaveOneOut


## Functions

In [ ]:
!git clone https://github.com/domingomery/balu3
!pip install ./balu3
print('Balu3 installed.')

Cloning into 'balu3'...
remote: Enumerating objects: 267, done.
remote: Counting objects: 100% (267/267), done.
remote: Compressing objects: 100% (188/188), done.
remote: Total 267 (delta 155), reused 181 (delta 75), pack-reused 0
Receiving objects: 100% (267/267), 70.23 KiB | 1.03 MiB/s, done.
Resolving deltas: 100% (155/155), done.
Processing ./balu3
  Preparing metadata (setup.py) ... done
  Created wheel for balu3: filename=balu3-1.0-py3-none-any.whl size=43700 sha256=b485ab96fdbbaca4f96f2c4ab86dd8051a81652175af31fa960e4de1a0ce4e45
  Stored in directory: /tmp/pip-ephem-wheel-cache-hw9f_ceb/wheels/9e/f6/a9/3d39da83a3504331129d706f5b04ceb3dbf0ec0f313cdb8fcf
Successfully built balu3
Balu3 installed.


In [ ]:
from   balu3.fs.sel    import sfs,clean
from   balu3.ft.norm   import minmax

# Class for SFS
class sfsbalu():
  def __init__(self, p, show=0):
    self.p = p
    self.show = show
  def fit(self,X,y):
    self.sel = sfs(X,y,self.p,show=self.show)
    self.m   = X.shape[1]
  def get_support(self):
    s = np.zeros((self.m,))
    s[self.sel] = 1
    return s

def holdout(clf,Xtrain,ytrain,Xtest,ytest,show=1):
  clf.fit(Xtrain, ytrain)
  ypred  = clf.predict(Xtest)
  acc = accuracy_score(ytest,ypred)
  C   = confusion_matrix(ytest,ypred)
  if show:
    acc_st = "{:.2f}".format(acc*100)
    print('Accuracy = '+str(acc_st))
    print('Confusion Matrix:')
    print(C)
  return acc,C,ypred

def crossval(clf,X,y,nfolds=10,show=1):
  scores = cross_val_score(clf, X, y, cv=nfolds)
  acc = np.mean(scores)
  if show:
    acc_st = "{:.2f}".format(acc*100)
    print('Accuracy = '+str(acc_st))
  return acc


##  New Data

In [ ]:
!wget https://www.dropbox.com/s/5sj3y39pm9uuj8w/Features_Letras.zip
!unzip -qq Features_Letras

--2023-07-21 15:24:23--  https://www.dropbox.com/s/5sj3y39pm9uuj8w/Features_Letras.zip
Resolving www.dropbox.com (www.dropbox.com)... 162.125.64.18, 2620:100:6035:18::a27d:5512
Connecting to www.dropbox.com (www.dropbox.com)|162.125.64.18|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: /s/raw/5sj3y39pm9uuj8w/Features_Letras.zip [following]
--2023-07-21 15:24:24--  https://www.dropbox.com/s/raw/5sj3y39pm9uuj8w/Features_Letras.zip
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc9197e31b95ba818fec27f5b125.dl.dropboxusercontent.com/cd/0/inline/CASXKHa1Yq6vghH2LDsSJ6r9Z_kerUJY-zuw3D_ziTQSvjHidUC_LWT8TPcXInhTSP9zBDTHCCc0R7-gFgcbFyzRptU8gzyQlrrXW2ePOSbu7bhsS3J7Vhc24XOPzy26CBK9mYlA6GfRu9eRknC15Z-Y8FIyIwXWqpL6iOdEfTJYWQ/file# [following]
--2023-07-21 15:24:25--  https://uc9197e31b95ba818fec27f5b125.dl.dropboxusercontent.com/cd/0/inline/CASXKHa1Yq6vghH2LDsSJ6r9Z_kerUJY-zuw3D_ziTQSvjHidUC_LWT8

In [ ]:
X = np.load('Features_Letras/Letras_X.npy')            # features
y = np.load('Features_Letras/Letras_y.npy')            # labels
y = y.astype(int)
print('Features: '+str(X.shape[0]) +' samples with '+str(X.shape[1]) +' features')
print(' Classes: '+str(int(np.min(y)))+'...'+str(int(np.max(y))))

Features: 500 samples with 45 features
 Classes: 0...4


## Spliting Training/Testing

... + Clean + Normalización

In [ ]:
Xtrain, Xtest, ytrain, ytest = train_test_split(X, y, shuffle=True, stratify=y, test_size=0.10)
sclean = clean(Xtrain)            # índice de características seleccionadas
Xtrain_clean = Xtrain[:,sclean]   # nuevo training
Xtest_clean  = Xtest[:,sclean]    # nuevo testing
Xtrain, a, b = minmax(Xtrain_clean)
Xtest        = Xtest_clean * a + b

print('Training: '+str(Xtrain.shape[0]) +' samples with '+str(Xtrain.shape[1]) +' features')
print(' Testing: '+str(Xtest.shape[0]) +' samples with '+str(Xtest.shape[1]) +' features')


Training: 450 samples with 41 features
 Testing: 50 samples with 41 features


## List of Feature Selectors

There are $n$ selectors $(g_1, \cdots g_n)$:

In [ ]:
from sklearn.feature_selection import SelectKBest, chi2
from sklearn.feature_selection import SelectFromModel
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.feature_selection import SelectFromModel
from sklearn.feature_selection import RFE
from sklearn.svm import SVC

nsel = 8 # number of features to be selected

g = [sfsbalu(p=nsel),
     SelectFromModel(RandomForestClassifier(n_estimators=100), max_features=nsel),
     SelectFromModel(LogisticRegression(penalty="l2"), max_features=nsel)
     ]

n = len(g)


## List of Classifiers

There are $q$ classifiers $(h_1, \cdots h_q)$:

In [ ]:
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis

h = [SVC(kernel = "linear"),
     SVC(kernel = "rbf", gamma=0.5,C=0.3),
     KNeighborsClassifier(n_neighbors=5),
     KNeighborsClassifier(n_neighbors=7),
     LinearDiscriminantAnalysis(),
     ]

q = len(h)

## Evaluation of selector $g_i$ using  classifier $h_k$

In [ ]:
# Selecctor g[i]
i = 1
model = g[i]
model.fit(Xtrain, ytrain)
sel = np.nonzero(model.get_support())[0]
print('selector g['+str(i)+']:'+str(sel)+'<= columnas seleccionadas')

selector g[1]:[ 5 13 14 15 30 32 36 40]<= columnas seleccionadas


In [ ]:
# Hold-out with classifier h[k]
k = 3
clf = h[k]
print('Clasificador h['+str(k)+']:')
acc,C,_ = holdout(clf,Xtrain[:,sel],ytrain,Xtest[:,sel],ytest)


Clasificador h[3]:
Accuracy = 100.00
Confusion Matrix:
[[10  0  0  0  0]
 [ 0 10  0  0  0]
 [ 0  0 10  0  0]
 [ 0  0  0 10  0]
 [ 0  0  0  0 10]]


... with these functions it is very easy to implement strategies that look for the best combination $(g_i,h_k)$, not forgetting that to evaluate the accuracy $(\eta)$ it is possible to use `holdout` or `crossval`.